In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dynesty import NestedSampler
from dynesty import plotting as dyplot

import warnings

warnings.filterwarnings('ignore')

%matplotlib widget

In [ ]:
# # Simulated data for demonstration
# np.random.seed(42)
# true_params = {'a': 2.0, 't0': 58804.5, 'n': 1.5, 'c': 1.0}
# t = np.linspace(58802, 58807, 100)
# y = np.piecewise(t,
#                  [t < true_params['t0'], t >= true_params['t0']],
#                  [lambda t: true_params['c'],
#                   lambda t: true_params['a'] * (t - true_params['t0'])**true_params['n'] + true_params['c']])
# yerr = 0.5
# y_obs = y + np.random.normal(0, yerr, size=len(t))

In [ ]:
og_lc = np.load('/Users/zgl12/Modules/SN2019vxm/Data/tess_sector_18_psf.npy')

t = og_lc[0][og_lc[0] >= 58802.5]
y_obs = og_lc[1][og_lc[0] >= 58802.5]
y_err = og_lc[2][og_lc[0] >= 58802.5]

print(np.nansum(~np.isfinite(t)))

dy = 1

y_err[y_err < dy] = dy

plt.figure()
plt.errorbar(t, y_obs, y_err, fmt = '.')
plt.show()

def residuals(theta, mjd, flux, flux_err):
    
    mjds = mjd[mjd >= theta[3]]
    fluxes = flux[mjd >= theta[3]]
    flux_errs = flux_err[mjd >= theta[3]]
    
    model = power_law(mjds, *theta)
    
    resid = np.nansum((abs(model - fluxes))**2 / flux_errs**2 ) / (len(fluxes) - len(theta) - 1)
    if (resid == 0) | (np.isnan(resid)):
        return np.nan
    else:
        return resid

def power_law(x, a, n, c, d): #, f):
    # d = x[0]
    # return a * (x-d)**(n*(x-d) + f) + c
    return a * (x-d)**(n) + c


residuals([3.02, 1.38, -0.20, 58804.27], t, y_obs, y_err), residuals([2.74, 1.41, -0.79, 58804.03], t, y_obs, y_err)

# t
# y_ob
# yerr

In [ ]:
# Model definition
def model(theta, t):
    a, t0, n, c = theta
    return np.where(t < t0, c, a * (t - t0)**n + c)

# Log-likelihood function
def log_likelihood(theta):
    y_model = model(theta, t)
    return -0.5 * np.nansum(((y_obs - y_model) / y_err)**2) #/ (len(t[t>=theta[1]]) - 5)

# Prior transform from unit cube to parameter space
def prior_transform(u):
    ua, ut0, un, uc = u
    a = 1 + 3 * ua         # Uniform(0, 5)
    t0 = 58803.5 + 1.5 * ut0   # Uniform(58803, 58806)
    n = 1.2 + 0.5 * un     # Uniform(1.1, 2)
    c = -2 + 4 * uc       # Uniform(-5, 5)
    return a, t0, n, c

In [ ]:
# Run dynesty nested sampling
dsampler = NestedSampler(log_likelihood, prior_transform, ndim=4, nlive=2000, )
dsampler.run_nested(dlogz=1e-4)
results = dsampler.results

# Plotting posterior distributions
fig, axes = dyplot.cornerplot(results, show_titles=True,
                              title_fmt='.2f', labels=['a', 't0', 'n', 'c'])

plt.show()

In [1]:
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.coordinates import SkyCoord

# Optional: set Vizier row limit (None for all rows, or an integer limit)
Vizier.ROW_LIMIT = -1  # return all rows

# Search for the XRBcat catalog by keyword or catalog ID
catalog_list = Vizier.find_catalogs("Xrbcat")

In [5]:
catalog = Vizier.get_catalogs("J/A+A/675/A199")

# J/A+A/677/A134

In [7]:
catalog[0]

Name,RA_ICRS,DE_ICRS,PosErr,XrayType,Porb,Ppulse,SpType,Vmag,Jmag,W1mag,Fsoftmin,Fsoftmax,Fhardmin,Fhardmax,MeanDist,Mx,Mopt,tMSPFlag,fchart,SimbadName
,deg,deg,arcsec,,d,s,,mag,mag,mag,1e-15 W / m2,1e-15 W / m2,1e-15 W / m2,1e-15 W / m2,pc,solMass,solMass,,,
str23,float64,float64,float64,str24,float64,float64,str10,float32,float32,float32,float64,float64,float64,float64,float64,float32,float32,uint8,str27,str23
NGC 104-X5,6.00407530000,-72.08147450000,1.019,"GC, XT, QNS",0.361100,--,,21.700,--,--,--,0.22008,--,--,4530.00000000,1.400,--,--,NGC_104-X5.pdf,[HJV94] 5
NGC 104-X7,6.01455833333,-72.08109444444,1.0041,"GC, XT, QNS",0.230000,--,,--,--,--,--,0.39397,--,--,4530.00000000,1.400,--,--,NGC_104-X7.pdf,[HJV94] 7
NGC 104-X9,6.01776666667,-72.08280277778,0.10127,"XT, BH?, RP, GC",0.019569,--,WD?,19.772,--,--,0.4415,4.4522,--,--,4530.00000000,--,--,--,NGC_104-X9.pdf,[HJV94] 9
NGC 104-W37,6.02086250000,-72.08090833333,1.0189,"GC, QNS",0.128600,--,,12.960,--,--,--,0.02568,--,--,4530.00000000,1.400,--,--,NGC_104-W37.pdf,[GHE2001] W37
IGR J00291+5934,7.26267520000,59.57194510000,1.0083,"XP, XT, RP",0.102500,0.00167,,--,--,--,0.038091,244.44,23.309,23.309,3100.00000000,1.400,0.100,--,IGR_J00291+5934.pdf,IGR J00291+5934
3U 0042+32,11.21049430000,33.01922320000,1.0097,XT,11.600000,--,G?,19.300,--,--,--,259,--,5.73,5000.00000000,--,--,--,3U_0042+32.pdf,3U 0042+32
4FGL J0407.7-5702,61.88217564450,-57.00701993499,0.00065543,XP?,--,--,,--,--,18.016,0.43896,0.49606,--,--,7000.00000000,1.400,--,2,4FGL_J0407.7-5702.pdf,4FGL J0407.7-5702
GRO J0422+32,65.42801144723,32.90748309894,0.010044,"BH, XT, RP, PSH",0.212167,--,M1-M5,13.200,--,16.634,0.0063893,4800,--,72000,2490.00000000,2.700,0.460,--,GRO_J0422+32.pdf,GRO J0422+32
